In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark = SparkSession.builder.getOrCreate()

In [12]:
# 아직 실행되지 않고, 객체로 생성되기만 한 상태임 
df = spark.\
    read.\
    csv("hdfs://192.168.0.160:8020/encore/20240614/civilian_force.csv", header=True)

In [13]:
# 실행 계획을 보는 함수
df.explain()

== Physical Plan ==
FileScan csv [date#284,values#285,realtime_start#286,realtime_end#287,state#288,id#289,title#290,frequency_short#291,units_short#292,seasonal_adjustment_short#293] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://192.168.0.160:8020/encore/20240614/civilian_force.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<date:string,values:string,realtime_start:string,realtime_end:string,state:string,id:string...




In [14]:
# 스파크에서 사용하는 데이터프레임
df.show()

+----------+--------+--------------+------------+------+--------------------+--------------------+---------------+-----------+-------------------------+
|      date|  values|realtime_start|realtime_end| state|                  id|               title|frequency_short|units_short|seasonal_adjustment_short|
+----------+--------+--------------+------------+------+--------------------+--------------------+---------------+-----------+-------------------------+
|1976-01-01|164014.0|    2023-01-21|  2023-01-21|Alaska|LAUST020000000000...|Civilian Labor Fo...|              A|    Persons|                      NSA|
|1977-01-01|173460.0|    2023-01-21|  2023-01-21|Alaska|LAUST020000000000...|Civilian Labor Fo...|              A|    Persons|                      NSA|
|1978-01-01|182114.0|    2023-01-21|  2023-01-21|Alaska|LAUST020000000000...|Civilian Labor Fo...|              A|    Persons|                      NSA|
|1979-01-01|184638.0|    2023-01-21|  2023-01-21|Alaska|LAUST020000000000...|Civil

In [15]:
# pandas의 info() 에 해당하는 함수로, 각 컬럼이 어떤 데이터 타입인지 보여줌 
df.schema

StructType([StructField('date', StringType(), True), StructField('values', StringType(), True), StructField('realtime_start', StringType(), True), StructField('realtime_end', StringType(), True), StructField('state', StringType(), True), StructField('id', StringType(), True), StructField('title', StringType(), True), StructField('frequency_short', StringType(), True), StructField('units_short', StringType(), True), StructField('seasonal_adjustment_short', StringType(), True)])

In [16]:
# inferSchema 옵션으로 내가 일일이 데이터 형식 지정하지 않아도 추정해서 schema 만들도록 설정
df2 = spark.\
    read.\
    csv("hdfs://192.168.0.160:8020/encore/20240614/civilian_force.csv", header=True, inferSchema=True)

In [18]:
# df와 달리 DoubleType, DateType 등으로 추론해준 부분이 있다. 
df2.schema

StructType([StructField('date', DateType(), True), StructField('values', DoubleType(), True), StructField('realtime_start', DateType(), True), StructField('realtime_end', DateType(), True), StructField('state', StringType(), True), StructField('id', StringType(), True), StructField('title', StringType(), True), StructField('frequency_short', StringType(), True), StructField('units_short', StringType(), True), StructField('seasonal_adjustment_short', StringType(), True)])

# RDD 개념

In [20]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [21]:
rdd1 = spark.sparkContext.textFile("file:///home/kafka/test.txt")

In [22]:
rdd1_2 = rdd1.flatMap(lambda x: x.split(" "))

In [24]:
rdd1_3 = rdd1_2.map(lambda x: (x, 1))

In [25]:
rdd1_4 = rdd1_3.reduceByKey(lambda a, b : a+b)

In [26]:
rdd1_5 = rdd1_4.map(lambda x: (x[1], x[0])).sortByKey()

In [27]:
print(rdd1_5.collect())

[(9, 'Project'), (9, 'Gutenberg’s'), (18, 'Alice’s'), (18, 'in'), (18, 'Lewis'), (18, 'Carroll'), (18, 'Adventures'), (18, 'Wonderland'), (18, 'by'), (27, 'is'), (27, 'use'), (27, 'of'), (27, 'anyone'), (27, 'anywhere'), (27, 'at'), (27, 'no'), (27, 'This'), (27, 'eBook'), (27, 'for'), (27, 'the'), (27, 'cost'), (27, 'and'), (27, 'with')]


In [28]:
# 위 과정을 변수 없이 보통 아래와 같이 씀 

print(spark.sparkContext.textFile("file:///home/kafka/test.txt").\
        flatMap(lambda x: x.split(" ")).\
        map(lambda x: (x, 1)).\
        reduceByKey(lambda a, b : a+b).\
        map(lambda x: (x[1], x[0])).sortByKey().\
        collect())

[(9, 'Project'), (9, 'Gutenberg’s'), (18, 'Alice’s'), (18, 'in'), (18, 'Lewis'), (18, 'Carroll'), (18, 'Adventures'), (18, 'Wonderland'), (18, 'by'), (27, 'is'), (27, 'use'), (27, 'of'), (27, 'anyone'), (27, 'anywhere'), (27, 'at'), (27, 'no'), (27, 'This'), (27, 'eBook'), (27, 'for'), (27, 'the'), (27, 'cost'), (27, 'and'), (27, 'with')]


# 예제 2

In [30]:
columns = ["Seqno","Quote"]
data = [("1", "Be the change that you wish to see in the world"),
    ("2", "Everyone thinks of changing the world, but no one thinks of changing himself."),
    ("3", "The purpose of our lives is to be happy."),
    ("4", "Be cool.")]
df = spark.createDataFrame(data,columns)
df.show(truncate=False) # 잘림 없이 끝까지 보기

+-----+-----------------------------------------------------------------------------+
|Seqno|Quote                                                                        |
+-----+-----------------------------------------------------------------------------+
|1    |Be the change that you wish to see in the world                              |
|2    |Everyone thinks of changing the world, but no one thinks of changing himself.|
|3    |The purpose of our lives is to be happy.                                     |
|4    |Be cool.                                                                     |
+-----+-----------------------------------------------------------------------------+



In [31]:
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
rdd = spark.sparkContext.parallelize(dept)
df = rdd.toDF()

In [32]:
df.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



In [33]:
df.show()

+---------+---+
|       _1| _2|
+---------+---+
|  Finance| 10|
|Marketing| 20|
|    Sales| 30|
|       IT| 40|
+---------+---+



In [34]:
# 컬럼명 만들어서 넣어주기 
deptColumns = ["dept_name", "dept_id"]
df2 = rdd.toDF(deptColumns)
df2.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



# 예제 3

# 사용하지 않을 건데 전체 패키지 불러오면 예기치 못한 오류 발생할 수 있음. 필요한 것만 불러와서 쓰기! 
from pyspark.sql import *

In [38]:
from pyspark.sql.types import StructType, StructField, StringType

deptSchema = StructType([
    StructField('dept_name', StringType(), True),
    StructField('dept_id', StringType(), True)
])

deptDF1 = spark.createDataFrame(rdd, schema = deptSchema)
deptDF1.printSchema()
deptDF1.show(truncate = False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: string (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



# 예제 4

In [39]:
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

# Column names
columns = ["firstname","lastname","country","state"]

# Create DataFrame
df = spark.createDataFrame(data = data, schema = columns)
df.show(truncate=False)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |Williams|USA    |CA   |
|Maria    |Jones   |USA    |FL   |
+---------+--------+-------+-----+



In [40]:
df.first()

Row(firstname='James', lastname='Smith', country='USA', state='CA')

## 컬럼 선택

In [41]:
df.select("firstname").first()

Row(firstname='James')

In [42]:
df.select("firstname", "lastname").show()

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|Williams|
|    Maria|   Jones|
+---------+--------+



In [43]:
df.select(df["firstname"], df["lastname"]).show(3)
df.select(df.firstname, df.lastname).show(3)

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|Williams|
+---------+--------+
only showing top 3 rows

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|Williams|
+---------+--------+
only showing top 3 rows



In [44]:
df.select(df.columns[:3]).show()

+---------+--------+-------+
|firstname|lastname|country|
+---------+--------+-------+
|    James|   Smith|    USA|
|  Michael|    Rose|    USA|
|   Robert|Williams|    USA|
|    Maria|   Jones|    USA|
+---------+--------+-------+



# 예제 5

In [45]:
data = [
        (("James",None,"Smith"),"OH","M"),
        (("Anna","Rose",""),"NY","F"),
        (("Julia","","Williams"),"OH","F"),
        (("Maria","Anne","Jones"),"NY","M"),
        (("Jen","Mary","Brown"),"NY","M"),
        (("Mike","Mary","Williams"),"OH","M")
        ]

from pyspark.sql.types import StructType,StructField, StringType        
schema = StructType([
    StructField('name', StructType([
         StructField('firstname', StringType(), True),
         StructField('middlename', StringType(), True),
         StructField('lastname', StringType(), True)
         ])),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True)
     ])
df2 = spark.createDataFrame(data = data, schema = schema)

In [46]:
df2.show()

+--------------------+-----+------+
|                name|state|gender|
+--------------------+-----+------+
|{James, null, Smith}|   OH|     M|
|      {Anna, Rose, }|   NY|     F|
| {Julia, , Williams}|   OH|     F|
|{Maria, Anne, Jones}|   NY|     M|
|  {Jen, Mary, Brown}|   NY|     M|
|{Mike, Mary, Will...|   OH|     M|
+--------------------+-----+------+



In [47]:
df2.first()

Row(name=Row(firstname='James', middlename=None, lastname='Smith'), state='OH', gender='M')

In [48]:
df2.select("name").show(3, truncate=False)

+--------------------+
|name                |
+--------------------+
|{James, null, Smith}|
|{Anna, Rose, }      |
|{Julia, , Williams} |
+--------------------+
only showing top 3 rows



In [49]:
df2.select("name.firstname", "name.lastname", "gender").show(3, truncate=False)

+---------+--------+------+
|firstname|lastname|gender|
+---------+--------+------+
|James    |Smith   |M     |
|Anna     |        |F     |
|Julia    |Williams|F     |
+---------+--------+------+
only showing top 3 rows



In [50]:
df2.select("name.*", "gender").show(3)

+---------+----------+--------+------+
|firstname|middlename|lastname|gender|
+---------+----------+--------+------+
|    James|      null|   Smith|     M|
|     Anna|      Rose|        |     F|
|    Julia|          |Williams|     F|
+---------+----------+--------+------+
only showing top 3 rows



# 예제 6

In [51]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

# Create DataFrame
schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)


root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [52]:
from pyspark.sql.functions import sum, avg, max

In [53]:
# 우리가 작업하는 네임스페이스 안에 겹치는 이름이 있으면 둘 중 하나는 포기해야 함 (python sum VS sql sum)
# del sum 하면 pyspark의 sum이 네임스페이스에서 삭제되고 python의 sum으로 다시 돌아온다 
sum

<function pyspark.sql.functions.sum(col: 'ColumnOrName') -> pyspark.sql.column.Column>

In [54]:
tmp = df.groupby("department").\
        agg(sum('salary').alias('sum_salary'),\
        avg('salary'))

In [55]:
# tmp 라는 df를 만들 계획
tmp.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[department#626], functions=[sum(salary#628L), avg(salary#628L)])
   +- Exchange hashpartitioning(department#626, 200), ENSURE_REQUIREMENTS, [plan_id=346]
      +- HashAggregate(keys=[department#626], functions=[partial_sum(salary#628L), partial_avg(salary#628L)])
         +- Project [department#626, salary#628L]
            +- Scan ExistingRDD[employee_name#625,department#626,state#627,salary#628L,age#629L,bonus#630L]




In [56]:
# 실제로 액션 수행
tmp.show()

+----------+----------+-----------------+
|department|sum_salary|      avg(salary)|
+----------+----------+-----------------+
|     Sales|    257000|85666.66666666667|
|   Finance|    351000|          87750.0|
| Marketing|    171000|          85500.0|
+----------+----------+-----------------+

